In [ ]:
!pip install CoolProp
!pip install pyswarms

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 2.7 MB/s eta 0:00:00


In [ ]:
import CoolProp.CoolProp as CP
import numpy as np
import pyswarms as ps

# Valores fixados
refri1 = 'R717'  # Ciclo superior
refri2 = 'R134a'  # Ciclo inferior
carga_termica = 1222  # Carga térmica em watts
delta_T = 5  # Diferença de temperatura no trocador
T_cond = 50 + 273.15  # Temperatura de condensação do ciclo 1 (50°C)
efi = 0.7
superaquecimento = 5  # Superaquecimento fixo em 5 °C

# Função para calcular o COP e variáveis adicionais
def calcular_COP_e_UA(P_3, P_sep):
    # Estado 1
    Temperatura_1 = -25 + 273.15
    h_1 = CP.PropsSI('H', 'T', Temperatura_1, 'Q', 1, refri2)
    P_1 = CP.PropsSI('P', 'T', Temperatura_1, 'Q', 1, refri2)

    # Estado 2
    P_2 = P_1
    Temperatura_2 = Temperatura_1 + superaquecimento
    h_2 = CP.PropsSI('H', 'T', Temperatura_2, 'P', P_2, refri2)
    s_2 = CP.PropsSI('S', 'T', Temperatura_2, 'P', P_2, refri2)

    # Estado 3
    s_3s = s_2
    h_3s = CP.PropsSI('H', 'S', s_3s, 'P', P_3, refri2)
    h_3 = ((h_3s - h_2) / efi) + h_2
    Temperatura_3 = CP.PropsSI('T', 'H', h_3, 'P', P_3, refri2)
    s_3 = CP.PropsSI('S', 'H', h_3, 'P', P_3, refri2)

    # Estado 4
    h_4 = CP.PropsSI('H', 'P', P_3, 'Q', 0, refri2)
    Temperatura_4 = CP.PropsSI('T', 'P', P_3, 'Q', 0, refri2)
    s_4 = CP.PropsSI('S', 'P', P_3, 'Q', 0, refri2)

    # Estado 5
    h_5 = h_1 + h_4 - h_2

    # Estado 6
    h_6 = h_5

    # Estado 7
    Temperatura_7 = Temperatura_4 - delta_T
    h_7 = CP.PropsSI('H', 'T', Temperatura_7, 'Q', 1, refri1)
    s_7 = CP.PropsSI('S', 'T', Temperatura_7, 'Q', 1, refri1)
    P_7 = CP.PropsSI('P', 'T', Temperatura_7, 'Q', 1, refri1)

    # Estado 8
    P_8 = P_sep
    s_8s = s_7
    h_8s = CP.PropsSI('H', 'P', P_8, 'S', s_8s, refri1)
    h_8 = ((h_8s - h_7) / efi) + h_7

    # Estado 9
    P_9 = P_sep
    h_9 = CP.PropsSI('H', 'P', P_9, 'Q', 0, refri1)

    # Estado 10
    h_10 = h_9
    Temperatura_10 = CP.PropsSI('T', 'P', P_7, 'H', h_10, refri1)
    s_10 = CP.PropsSI('S', 'P', P_7, 'H', h_10, refri1)

    # Estado 11
    P_11 = P_sep
    h_11 = CP.PropsSI('H', 'P', P_11, 'Q', 1, refri1)
    s_11 = CP.PropsSI('S', 'P', P_11, 'Q', 1, refri1)

    # Estado 13
    P_13 = CP.PropsSI('P', 'T', T_cond, 'Q', 0, refri1)
    h_13 = CP.PropsSI('H', 'T', T_cond, 'Q', 0, refri1)

    # Estado 12
    P_12 = P_13
    h_12s = CP.PropsSI('H', 'P', P_12, 'S', s_11, refri1)
    h_12 = ((h_12s - h_11) / efi) + h_11

    # Estado 14
    h_14 = h_13

    # Cálculo das vazões mássicas
    m1 = carga_termica / (h_1 - h_6)
    m2 = m1 * ((h_3 - h_4) / (h_7 - h_10))
    m3 = m2 * ((h_8 - h_9) / (h_11 - h_14))

    # Trabalhos e COP
    w1 = m1 * (h_3 - h_2)
    w2 = m2 * (h_8 - h_7)
    w3 = m3 * (h_12 - h_11)
    COP = carga_termica / (w1 + w2 + w3)

    # Cálculo do UA
    delta_T_m = (((Temperatura_3 - Temperatura_10) - (Temperatura_4 - Temperatura_7)) / np.log((Temperatura_3 - Temperatura_10) / (Temperatura_4 - Temperatura_7)))
    Q = m2 * (h_7 - h_10)
    UA = Q / delta_T_m

    # Informações do ciclo
    t0 = 30 + 273.15
    tl = -16 + 273.15
    ADTotal = (w1+w2+w3)+(carga_termica*(1-(t0/tl)))

    return COP, UA, ADTotal

# Função objetivo para o PSO
def funcao_objetivo(x):
    P_3, P_sep = x[:, 0], x[:, 1]
    COP_valores = np.array([-calcular_COP_e_UA(P_3[i], P_sep[i])[0] for i in range(len(P_3))])
    return COP_valores

# Limites de busca
P3_min, P3_max = 106399, 650000  # Limite para P3
Psep_min, Psep_max = 119374, 2032972  # Limite para Psep

# Configuração do PSO
opcoes = {'c1': 2, 'c2': 2, 'w': 1}
limites = (np.array([P3_min, Psep_min]), np.array([P3_max, Psep_max]))
optimizer = ps.single.GlobalBestPSO(n_particles=15, dimensions=2, options=opcoes, bounds=limites)

# Execução do PSO
melhor_custo, melhor_posicao = optimizer.optimize(funcao_objetivo, iters=40)

# Resultados
P3_otimizado, Psep_otimizado = melhor_posicao
COP_otimizado, UA_otimizado, ADtot = calcular_COP_e_UA(P3_otimizado, Psep_otimizado)

print(f"Resultados ótimos:")
print(f"P3 = {P3_otimizado / 1000:.2f} kPa")
print(f"Psep = {Psep_otimizado / 1000:.2f} kPa")
print(f"COP máximo = {COP_otimizado:.4f}")
print(f"UA correspondente = {UA_otimizado:.4f} W/K")
print(f"ADtot = {ADtot / 1:.2f} W")

2025-03-18 03:36:50,259 - pyswarms.single.global_best - INFO - Optimize for 40 iters with {'c1': 2, 'c2': 2, 'w': 1}
pyswarms.single.global_best: 100%|██████████|40/40, best_cost=-1.81
2025-03-18 03:37:04,813 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -1.8121363945528677, best pos: [320059.61937244 934425.12088705]


Resultados ótimos:
P3 = 320.06 kPa
Psep = 934.43 kPa
COP máximo = 1.8121
UA correspondente = 109.6925 W/K
ADtot = 455.75 W


In [ ]:
import CoolProp.CoolProp as CP
import numpy as np
import pyswarms as ps

# Valores fixados
refri1 = 'R717'  # Ciclo superior
refri2 = 'R134a'  # Ciclo inferior
carga_termica = 1222  # Carga térmica em watts
delta_T = 5  # Diferença de temperatura no trocador
T_cond = 50 + 273.15  # Temperatura de condensação do ciclo 1 (50°C)
efi = 0.7
superaquecimento = 5  # Superaquecimento fixo em 5 °C
P_3 = 320060
P_sep = 934430

# Estado 0
t0 = 30 + 273.15  # Temperatura ambiente (K)
p0 = 101325  # Pressão ambiente (Pa)
h0_2 = CP.PropsSI('H', 'T', t0, 'P', p0, refri2)
s0_2 = CP.PropsSI('S', 'T', t0, 'P', p0, refri2)
h0_1 = CP.PropsSI('H', 'T', t0, 'P', p0, refri1)
s0_1 = CP.PropsSI('S', 'T', t0, 'P', p0, refri1)
tl = -16 + 273.15

# Estado 1
Temperatura_1 = -25 + 273.15
h_1 = CP.PropsSI('H', 'T', Temperatura_1, 'Q', 1, refri2)
P_1 = CP.PropsSI('P', 'T', Temperatura_1, 'Q', 1, refri2)
s_1 = CP.PropsSI('S', 'T', Temperatura_1, 'Q', 1, refri2)
af1 = (h_1 - h0_2) - (t0 * (s_1 - s0_2))

# Estado 2
P_2 = P_1
Temperatura_2 = Temperatura_1 + superaquecimento
h_2 = CP.PropsSI('H', 'T', Temperatura_2, 'P', P_2, refri2)
s_2 = CP.PropsSI('S', 'T', Temperatura_2, 'P', P_2, refri2)
af2 = (h_2 - h0_2) - (t0 * (s_2 - s0_2))

# Estado 3
s_3s = s_2
h_3s = CP.PropsSI('H', 'S', s_3s, 'P', P_3, refri2)
h_3 = ((h_3s - h_2) / efi) + h_2
Temperatura_3 = CP.PropsSI('T', 'H', h_3, 'P', P_3, refri2)
s_3 = CP.PropsSI('S', 'H', h_3, 'P', P_3, refri2)
af3 = (h_3 - h0_2) - (t0 * (s_3 - s0_2))

# Estado 4
h_4 = CP.PropsSI('H', 'P', P_3, 'Q', 0, refri2)
Temperatura_4 = CP.PropsSI('T', 'P', P_3, 'Q', 0, refri2)
s_4 = CP.PropsSI('S', 'P', P_3, 'Q', 0, refri2)
af4 = (h_4 - h0_2) - (t0 * (s_4 - s0_2))

# Estado 5
h_5 = h_1 + h_4 - h_2
P_5 = P_3
s_5 = CP.PropsSI('S', 'H', h_5, 'P', P_5, refri2)
af5 = (h_5 - h0_2) - (t0 * (s_5 - s0_2))

# Estado 6
h_6 = h_5
s_6 = CP.PropsSI('S', 'H', h_6, 'P', P_1, refri2)
af6 = (h_6 - h0_2) - (t0 * (s_6 - s0_2))

# Estado 7
Temperatura_7 = Temperatura_4 - delta_T
h_7 = CP.PropsSI('H', 'T', Temperatura_7, 'Q', 1, refri1)
s_7 = CP.PropsSI('S', 'T', Temperatura_7, 'Q', 1, refri1)
P_7 = CP.PropsSI('P', 'T', Temperatura_7, 'Q', 1, refri1)
af7 = (h_7 - h0_1) - (t0 * (s_7 - s0_1))

# Estado 8
P_8 = P_sep
s_8s = s_7
h_8s = CP.PropsSI('H', 'P', P_8, 'S', s_8s, refri1)
h_8 = ((h_8s - h_7) / efi) + h_7
s_8 = CP.PropsSI('S', 'H', h_8, 'P', P_8, refri1)
af8 = (h_8 - h0_1) - (t0 * (s_8 - s0_1))

# Estado 9
P_9 = P_sep
h_9 = CP.PropsSI('H', 'P', P_9, 'Q', 0, refri1)
s_9 = CP.PropsSI('S', 'P', P_9, 'Q', 0, refri1)
af9 = (h_9 - h0_1) - (t0 * (s_9 - s0_1))

# Estado 10
h_10 = h_9
Temperatura_10 = CP.PropsSI('T', 'P', P_7, 'H', h_10, refri1)
s_10 = CP.PropsSI('S', 'P', P_7, 'H', h_10, refri1)
af10 = (h_10 - h0_1) - (t0 * (s_10 - s0_1))

# Estado 11
P_11 = P_sep
h_11 = CP.PropsSI('H', 'P', P_11, 'Q', 1, refri1)
s_11 = CP.PropsSI('S', 'P', P_11, 'Q', 1, refri1)
af11 = (h_11 - h0_1) - (t0 * (s_11 - s0_1))

# Estado 13
P_13 = CP.PropsSI('P', 'T', T_cond, 'Q', 0, refri1)
h_13 = CP.PropsSI('H', 'T', T_cond, 'Q', 0, refri1)
s_13 = CP.PropsSI('S', 'T', T_cond, 'Q', 0, refri1)
af13 = (h_13 - h0_1) - (t0 * (s_13 - s0_1))

# Estado 12
s_12s = s_11
P_12 = P_13
h_12s = CP.PropsSI('H', 'P', P_12, 'S', s_12s, refri1)
h_12 = ((h_12s - h_11) / efi) + h_11
s_12 = CP.PropsSI('S', 'H', h_12, 'P', P_12, refri1)
af12 = (h_12 - h0_1) - (t0 * (s_12 - s0_1))

# Estado 14
h_14 = h_13
P_14 = P_sep
s_14 = CP.PropsSI('S', 'H', h_14, 'P', P_14, refri1)
af14 = (h_14 - h0_1) - (t0 * (s_14 - s0_1))

# Cálculo das vazões mássicas
m1 = carga_termica / (h_1 - h_6)
m2 = m1 * ((h_3 - h_4) / (h_7 - h_10))
m3 = m2 * ((h_8 - h_9) / (h_11 - h_14))

# Trabalhos e COP
w1 = m1 * (h_3 - h_2)
w2 = m2 * (h_8 - h_7)
w3 = m3 * (h_12 - h_11)
COP = carga_termica / (w1 + w2 + w3)

# Cálculo do UA
delta_T_m = (((Temperatura_3 - Temperatura_10) - (Temperatura_4 - Temperatura_7)) / np.log((Temperatura_3 - Temperatura_10) / (Temperatura_4 - Temperatura_7)))
Q = m2 * (h_7 - h_10)
UA = Q / delta_T_m

# Exergias destruidas
ADComp1 = (m1*af2)+w1-(m1*af3)
ADValv1 = m1*(af5-af6)
ADTroc = m1*(af1+af4-af2-af5)
ADEvap = (m1*af6)+(carga_termica*(1-(t0/tl)))-(m1*af1)
ADTrocCasc = (m1*(af3-af4))+(m2*(af10-af7))
ADComp2 = (m2*af7)+w2-(m2*af8)
ADValv2 = m2*(af9-af10)
ADSep = (m2*(af8-af9))+(m3*(af14-af11))
ADComp3 = (m3*af11)+w3-(m3*af12)
ADValv3 = m3*(af13-af14)
ADCond = m3*(af12-af13)

# Informações do ciclo
ADTotal = (w1+w2+w3)+(carga_termica*(1-(t0/tl)))
th = t0
COPteorico = tl / (th - tl)
N2 = COP / COPteorico

tot = ADComp1 + ADValv1 + ADTroc + ADEvap + ADTrocCasc + ADComp2 + ADValv2 + ADSep + ADComp3 + ADValv3 + ADCond

print(round(ADComp1,1))
print(round(ADTrocCasc,1))
print(round(ADTroc,1))
print(round(ADValv1,1))
print(round(ADEvap,1))
print(round(ADComp2,1))
print(round(ADSep,1))
print(round(ADValv2,1))
print(round(ADComp3,1))
print(round(ADValv3,1))
print(round(ADCond,1))


print(round(ADTotal,1))
print(ADComp1+ADTrocCasc+ADTroc+ADValv1+ADEvap+ADComp2+ADSep+ADValv2+ADComp3+ADValv3+ADCond)

67.3
34.8
2.7
13.4
52.2
56.8
15.0
8.5
61.9
12.4
130.6
455.7
455.7460181689348


In [ ]:
import matplotlib.pyplot as plt
import CoolProp.CoolProp as CP
import numpy as np

# Valores fixados
refri1 = 'R717'  # Ciclo superior
refri2 = 'R134a'  # Ciclo inferior
carga_termica = 1222  # Carga térmica em watts
delta_T = 5  # Diferença de temperatura no trocador
T_cond = 50 + 273.15  # Temperatura de condensação do ciclo 1 (50°C)
efi = 0.7
superaquecimento = 5

# Valores de Psep para as isolinhas em Pa
valores_de_Psep = [842730, 942730, 1042730]

# Intervalo de valores de P6 para o gráfico em Pa
valores_de_P2 = np.linspace(106399, 650000, 100)

# Loop sobre os valores de Psep (para as isolinhas)
for P_sep in valores_de_Psep:
    valores_COP = []
    pressoes_p2 = []

    # Loop para diferentes valores de P6
    for P_3 in valores_de_P2:

        # Estado 0
        t0 = 30 + 273.15  # Temperatura ambiente (K)
        p0 = 101325  # Pressão ambiente (Pa)
        h0_2 = CP.PropsSI('H', 'T', t0, 'P', p0, refri2)
        s0_2 = CP.PropsSI('S', 'T', t0, 'P', p0, refri2)
        h0_1 = CP.PropsSI('H', 'T', t0, 'P', p0, refri1)
        s0_1 = CP.PropsSI('S', 'T', t0, 'P', p0, refri1)
        tl = -16 + 273.15

        # Estado 1
        Temperatura_1 = -25 + 273.15
        h_1 = CP.PropsSI('H', 'T', Temperatura_1, 'Q', 1, refri2)
        P_1 = CP.PropsSI('P', 'T', Temperatura_1, 'Q', 1, refri2)
        s_1 = CP.PropsSI('S', 'T', Temperatura_1, 'Q', 1, refri2)
        af1 = (h_1 - h0_2) - (t0 * (s_1 - s0_2))

        # Estado 2
        P_2 = P_1
        Temperatura_2 = Temperatura_1 + superaquecimento
        h_2 = CP.PropsSI('H', 'T', Temperatura_2, 'P', P_2, refri2)
        s_2 = CP.PropsSI('S', 'T', Temperatura_2, 'P', P_2, refri2)
        af2 = (h_2 - h0_2) - (t0 * (s_2 - s0_2))

        # Estado 3
        s_3s = s_2
        h_3s = CP.PropsSI('H', 'S', s_3s, 'P', P_3, refri2)
        h_3 = ((h_3s - h_2) / efi) + h_2
        Temperatura_3 = CP.PropsSI('T', 'H', h_3, 'P', P_3, refri2)
        s_3 = CP.PropsSI('S', 'H', h_3, 'P', P_3, refri2)
        af3 = (h_3 - h0_2) - (t0 * (s_3 - s0_2))

        # Estado 4
        h_4 = CP.PropsSI('H', 'P', P_3, 'Q', 0, refri2)
        Temperatura_4 = CP.PropsSI('T', 'P', P_3, 'Q', 0, refri2)
        s_4 = CP.PropsSI('S', 'P', P_3, 'Q', 0, refri2)
        af4 = (h_4 - h0_2) - (t0 * (s_4 - s0_2))

        # Estado 5
        h_5 = h_1 + h_4 - h_2
        P_5 = P_3
        s_5 = CP.PropsSI('S', 'H', h_5, 'P', P_5, refri2)
        af5 = (h_5 - h0_2) - (t0 * (s_5 - s0_2))

        # Estado 6
        h_6 = h_5
        s_6 = CP.PropsSI('S', 'H', h_6, 'P', P_1, refri2)
        af6 = (h_6 - h0_2) - (t0 * (s_6 - s0_2))

        # Estado 7
        Temperatura_7 = Temperatura_4 - delta_T
        h_7 = CP.PropsSI('H', 'T', Temperatura_7, 'Q', 1, refri1)
        s_7 = CP.PropsSI('S', 'T', Temperatura_7, 'Q', 1, refri1)
        P_7 = CP.PropsSI('P', 'T', Temperatura_7, 'Q', 1, refri1)
        af7 = (h_7 - h0_1) - (t0 * (s_7 - s0_1))

        # Estado 8
        P_8 = P_sep
        s_8s = s_7
        h_8s = CP.PropsSI('H', 'P', P_8, 'S', s_8s, refri1)
        h_8 = ((h_8s - h_7) / efi) + h_7
        s_8 = CP.PropsSI('S', 'H', h_8, 'P', P_8, refri1)
        af8 = (h_8 - h0_1) - (t0 * (s_8 - s0_1))

        # Estado 9
        P_9 = P_sep
        h_9 = CP.PropsSI('H', 'P', P_9, 'Q', 0, refri1)
        s_9 = CP.PropsSI('S', 'P', P_9, 'Q', 0, refri1)
        af9 = (h_9 - h0_1) - (t0 * (s_9 - s0_1))

        # Estado 10
        h_10 = h_9
        Temperatura_10 = CP.PropsSI('T', 'P', P_7, 'H', h_10, refri1)
        s_10 = CP.PropsSI('S', 'P', P_7, 'H', h_10, refri1)
        af10 = (h_10 - h0_1) - (t0 * (s_10 - s0_1))

        # Estado 11
        P_11 = P_sep
        h_11 = CP.PropsSI('H', 'P', P_11, 'Q', 1, refri1)
        s_11 = CP.PropsSI('S', 'P', P_11, 'Q', 1, refri1)
        af11 = (h_11 - h0_1) - (t0 * (s_11 - s0_1))

        # Estado 13
        P_13 = CP.PropsSI('P', 'T', T_cond, 'Q', 0, refri1)
        h_13 = CP.PropsSI('H', 'T', T_cond, 'Q', 0, refri1)
        s_13 = CP.PropsSI('S', 'T', T_cond, 'Q', 0, refri1)
        af13 = (h_13 - h0_1) - (t0 * (s_13 - s0_1))

        # Estado 12
        s_12s = s_11
        P_12 = P_13
        h_12s = CP.PropsSI('H', 'P', P_12, 'S', s_12s, refri1)
        h_12 = ((h_12s - h_11) / efi) + h_11
        s_12 = CP.PropsSI('S', 'H', h_12, 'P', P_12, refri1)
        af12 = (h_12 - h0_1) - (t0 * (s_12 - s0_1))

        # Estado 14
        h_14 = h_13
        P_14 = P_sep
        s_14 = CP.PropsSI('S', 'H', h_14, 'P', P_14, refri1)
        af14 = (h_14 - h0_1) - (t0 * (s_14 - s0_1))

        # Cálculo das vazões mássicas
        m1 = carga_termica / (h_1 - h_6)
        m2 = m1 * ((h_3 - h_4) / (h_7 - h_10))
        m3 = m2 * ((h_8 - h_9) / (h_11 - h_14))

        # Trabalhos e COP
        w1 = m1 * (h_3 - h_2)
        w2 = m2 * (h_8 - h_7)
        w3 = m3 * (h_12 - h_11)
        COP = carga_termica / (w1 + w2 + w3)

        # Armazenar os resultados para plotagem
        valores_COP.append(COP)
        pressoes_p2.append(P_3 / 1e6)  # Convertendo P6 para kPa

    # Plotar o gráfico UA x P2 para cada valor de Psep
    plt.plot(pressoes_p2, valores_COP, label=f'Psep = {P_sep / 1e3} kPa')

# Configurações do gráfico
plt.xlabel('P3 (MPa)')
plt.ylabel('COP')
plt.legend()
plt.grid(False)
plt.show()

ModuleNotFoundError: No module named 'CoolProp'

In [ ]:
import matplotlib.pyplot as plt
import CoolProp.CoolProp as CP
import numpy as np

# Valores fixados
refri1 = 'R717'  # Ciclo superior
refri2 = 'R134a'  # Ciclo inferior
carga_termica = 1222  # Carga térmica em watts
delta_T = 5  # Diferença de temperatura no trocador
T_cond = 50 + 273.15  # Temperatura de condensação do ciclo superior (50°C)
efi = 0.7
T_evap = -25 + 273.15
superaquecimento = 5

# Valores de Psep para as isolinhas em Pa
valores_de_Psep = [842730, 942730, 1042730]

# Intervalo de valores de P6 para o gráfico em Pa
valores_de_P2 = np.linspace(106399, 650000, 100)

# Loop sobre os valores de Psep (para as isolinhas)
for P_sep in valores_de_Psep:
    valores_UA = []
    pressoes_p2 = []
    for P_3 in valores_de_P2:
        # Estado 0
        t0 = 30 + 273.15  # Temperatura ambiente (K)
        p0 = 101325  # Pressão ambiente (Pa)
        h0_2 = CP.PropsSI('H', 'T', t0, 'P', p0, refri2)
        s0_2 = CP.PropsSI('S', 'T', t0, 'P', p0, refri2)
        h0_1 = CP.PropsSI('H', 'T', t0, 'P', p0, refri1)
        s0_1 = CP.PropsSI('S', 'T', t0, 'P', p0, refri1)
        tl = -16 + 273.15

        # Estado 1
        Temperatura_1 = -25 + 273.15
        h_1 = CP.PropsSI('H', 'T', Temperatura_1, 'Q', 1, refri2)
        P_1 = CP.PropsSI('P', 'T', Temperatura_1, 'Q', 1, refri2)
        s_1 = CP.PropsSI('S', 'T', Temperatura_1, 'Q', 1, refri2)
        af1 = (h_1 - h0_2) - (t0 * (s_1 - s0_2))

        # Estado 2
        P_2 = P_1
        Temperatura_2 = Temperatura_1 + superaquecimento
        h_2 = CP.PropsSI('H', 'T', Temperatura_2, 'P', P_2, refri2)
        s_2 = CP.PropsSI('S', 'T', Temperatura_2, 'P', P_2, refri2)
        af2 = (h_2 - h0_2) - (t0 * (s_2 - s0_2))

        # Estado 3
        s_3s = s_2
        h_3s = CP.PropsSI('H', 'S', s_3s, 'P', P_3, refri2)
        h_3 = ((h_3s - h_2) / efi) + h_2
        Temperatura_3 = CP.PropsSI('T', 'H', h_3, 'P', P_3, refri2)
        s_3 = CP.PropsSI('S', 'H', h_3, 'P', P_3, refri2)
        af3 = (h_3 - h0_2) - (t0 * (s_3 - s0_2))

        # Estado 4
        h_4 = CP.PropsSI('H', 'P', P_3, 'Q', 0, refri2)
        Temperatura_4 = CP.PropsSI('T', 'P', P_3, 'Q', 0, refri2)
        s_4 = CP.PropsSI('S', 'P', P_3, 'Q', 0, refri2)
        af4 = (h_4 - h0_2) - (t0 * (s_4 - s0_2))

        # Estado 5
        h_5 = h_1 + h_4 - h_2
        P_5 = P_3
        s_5 = CP.PropsSI('S', 'H', h_5, 'P', P_5, refri2)
        af5 = (h_5 - h0_2) - (t0 * (s_5 - s0_2))

        # Estado 6
        h_6 = h_5
        s_6 = CP.PropsSI('S', 'H', h_6, 'P', P_1, refri2)
        af6 = (h_6 - h0_2) - (t0 * (s_6 - s0_2))

        # Estado 7
        Temperatura_7 = Temperatura_4 - delta_T
        h_7 = CP.PropsSI('H', 'T', Temperatura_7, 'Q', 1, refri1)
        s_7 = CP.PropsSI('S', 'T', Temperatura_7, 'Q', 1, refri1)
        P_7 = CP.PropsSI('P', 'T', Temperatura_7, 'Q', 1, refri1)
        af7 = (h_7 - h0_1) - (t0 * (s_7 - s0_1))

        # Estado 8
        P_8 = P_sep
        s_8s = s_7
        h_8s = CP.PropsSI('H', 'P', P_8, 'S', s_8s, refri1)
        h_8 = ((h_8s - h_7) / efi) + h_7
        s_8 = CP.PropsSI('S', 'H', h_8, 'P', P_8, refri1)
        af8 = (h_8 - h0_1) - (t0 * (s_8 - s0_1))

        # Estado 9
        P_9 = P_sep
        h_9 = CP.PropsSI('H', 'P', P_9, 'Q', 0, refri1)
        s_9 = CP.PropsSI('S', 'P', P_9, 'Q', 0, refri1)
        af9 = (h_9 - h0_1) - (t0 * (s_9 - s0_1))

        # Estado 10
        h_10 = h_9
        Temperatura_10 = CP.PropsSI('T', 'P', P_7, 'H', h_10, refri1)
        s_10 = CP.PropsSI('S', 'P', P_7, 'H', h_10, refri1)
        af10 = (h_10 - h0_1) - (t0 * (s_10 - s0_1))
        x10 = CP.PropsSI('Q', 'P', P_7, 'H', h_10, refri1)

        # Estado 11
        P_11 = P_sep
        h_11 = CP.PropsSI('H', 'P', P_11, 'Q', 1, refri1)
        s_11 = CP.PropsSI('S', 'P', P_11, 'Q', 1, refri1)
        af11 = (h_11 - h0_1) - (t0 * (s_11 - s0_1))

        # Estado 13
        P_13 = CP.PropsSI('P', 'T', T_cond, 'Q', 0, refri1)
        h_13 = CP.PropsSI('H', 'T', T_cond, 'Q', 0, refri1)
        s_13 = CP.PropsSI('S', 'T', T_cond, 'Q', 0, refri1)
        af13 = (h_13 - h0_1) - (t0 * (s_13 - s0_1))

        # Estado 12
        s_12s = s_11
        P_12 = P_13
        h_12s = CP.PropsSI('H', 'P', P_12, 'S', s_12s, refri1)
        h_12 = ((h_12s - h_11) / efi) + h_11
        s_12 = CP.PropsSI('S', 'H', h_12, 'P', P_12, refri1)
        af12 = (h_12 - h0_1) - (t0 * (s_12 - s0_1))

        # Estado 14
        h_14 = h_13
        P_14 = P_sep
        s_14 = CP.PropsSI('S', 'H', h_14, 'P', P_14, refri1)
        af14 = (h_14 - h0_1) - (t0 * (s_14 - s0_1))

        # Cálculo das vazões mássicas
        m1 = carga_termica / (h_1 - h_6)
        m2 = m1 * ((h_3 - h_4) / (h_7 - h_10))
        m3 = m2 * ((h_8 - h_9) / (h_11 - h_14))

        # Trabalhos e COP
        w1 = m1 * (h_3 - h_2)
        w2 = m2 * (h_8 - h_7)
        w3 = m3 * (h_12 - h_11)
        COP = carga_termica / (w1 + w2 + w3)

        # Cálculo do UA
        delta_T_m = (((Temperatura_3 - Temperatura_10) - (Temperatura_4 - Temperatura_7)) / np.log((Temperatura_3 - Temperatura_10) / (Temperatura_4 - Temperatura_7)))
        Q = m2 * (h_7 - h_10)
        UA = Q / delta_T_m

        # Armazenar os resultados para plotagem
        valores_UA.append(UA)
        pressoes_p2.append(P_3 / 1e6)

    plt.plot(pressoes_p2, valores_UA, label=f'Psep = {P_sep / 1e3} kPa')

# Configurações do gráfico
plt.xlabel('P3 (MPa)')
plt.ylabel('UA (W/K)')
plt.legend()
plt.grid(False)
plt.show()